<hr style="border: 20px solid black">

<h2>Question 1 (25 points) - Implement the First-Dispatch Routing Protocol with a Maximum Dispatch Radius</h2>
<br>
<font size="+1">
    <ul>
        <li style="color:blue">Recall:</li>
        <br>
        <ul style="color:blue">
            <li>A rider's request can be matched with a dispatchable driver using very simple algorithms, such as the <b><i>first-dispatch protocol</i></b>.</li>
            <br>
            <ul>
                <li>In the first-dispatch protocol only open drivers are considered as dispatchable.</li>
                <br>
                <li><b>Each request is immediately assigned to the open driver who is predicted to have the shortest en route time</b>.</li>
                <br>
                <li>This matching algorithm is also called the <i>on-call</i> policy, the <i>closest driver</i> policy, and the <i>committed driver-rider</i> matching policy.</li>
                <br>
            </ul>
            <li>When this committed driver-rider matching is used by ride-hailing platforms, the dispatched driver could miss the opportunity of being matched with a rider that they drive passed while en route to their matched rider.</li>
            <br>
            <li>To mitigate this inefficiency, some ride-hailing platforms proposed a <i>maximum dispatch radius</i> (MDR)  constraint with carefully selected thresholds.</li>
            <br>
            <li>Maximum dispatch radius means that a dispatch is made only if the en route time is below the threshold.</li>
            <br>
            <li>This prevents the driver from being dispatched to pick up very distant riders.</li>
            <br>
        </ul>
        <li>Assume you want to implement a benchmark matching algorithm solution that is capable of dispatching a single driver, from a collection of available drivers, to a single rider.</li>
        <br>
        <li>Specifically, you are to implement the so-called <i>first-dispatch protocol</i> as an optimization problem given the randomly generated data below.</li>
        <br>
        <li>The data consists of:</li>
        <br>
        <ul>
            <li>A single rider that is requesting a pick-up from a collection of drivers.</li>
            <br>
            <li>A data frame consisting of some measure of physical distance from the rider to each driver.</li>
            <br>
            <li>A maximum dispatch radius (MDR) that represents a cutoff value for drivers to be dispatched to riders.</li>
            <br>
            <ul>
                <li>In other words, if a driver's distance to the rider is larger than the MDR, then the driver cannot be dispatched to the rider's request.</li>
                <br>
            </ul>
            <li>A data frame consisting of the en route time, also known as the expected time of arrival (ETA), which represents the estimated time it will take for a driver to meet a rider once they have been dispatched.</li>
            <br>
        </ul>
        <li><b>Using the given data, formulate an optimization model (decision variables, objective function, constraints) that can be used to implement the <b style="color:blue">first-dispatch</b> protocol that also screens drivers according to the maximum dispatch radius constraint.</b></li>
        <br>
        <li>That is,</li>
        <br>
        <ul>
            <li>what decisions does the first-dispatch protocol have to make?</li>
            <br>
            <li>what is the goal or objective of the first-dispatch protocol?</li>
            <br>
            <li>what constraints, besides the MDR constraint, does the first-dispatch protocol have?</li>
            <br>
            <li>given the data below, how is the first-dispatch protocol suggesting the drivers be dispatched to the single rider?</li>
            <br>
        </ul>
        <li style="color:red"><b>You should display your formulation and solutions in an easy to read and interpret format. Your code should be easy to interpret, and if it is not, you should add sufficient commentary through comments, Markdown, or $\LaTeX$.</b></li>
        <br>
    </ul>
</font>

In [1]:
import numpy as np
import pandas as pd

import cvxpy as cp

In [2]:
rng = np.random.default_rng(42)

In [3]:
# Number of Riders
num_riders = 1

# Number of drivers
num_drivers = rng.poisson(lam=20)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 10, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 5 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta = distances.apply(lambda x: rng.pareto(x))# rng.exponential(x)) 

In [4]:
eta

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0.054216,0.011114,0.210143,0.07277,0.052392,0.174482,0.738491,0.054248,0.132412,0.071678,...,0.271868,7.841425,0.139952,0.011628,0.161365,0.465232,0.122537,0.031853,0.042093,0.088881


In [5]:
# distances = distances.transform(lambda x: x+20) # Sanity Check

distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,8.585979,6.97368,0.941773,9.756224,7.611397,7.860643,1.281136,4.503859,3.70798,9.26765,...,5.545848,0.638173,8.276312,6.316644,7.580877,3.54526,9.70698,8.931211,7.783835,1.946387


In [6]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0,0,1,0,0,0,1,1,1,0,...,0,1,0,0,0,1,0,0,0,1


**Decision Variables:**


MDR: Satisfy that driver is within the radius of 5

ETA: Pick the driver within MDR that has lowest eta, hence minimize

Driver: Match with riders

Rider: Match with drivers

**Objective function:**


Minimize the following:

f(x) = x1*eta + x2*eta + x3*eta + x4*eta +x5*eta + x6*eta + x7*eta....x24*eta

In [7]:
#Creating our variables, and keeping it dynamic to that of the DF columns

x = cp.Variable(len(eta.columns), integer = True)

In [8]:
#Transposing to perform matrix multiplication
time = np.array(eta).T

#Objective function
#Multiplies our variables with the time array
penalty = x @ time


#Minimizing the ETA
objective = cp.Minimize(penalty)

In [9]:
#Creating a matrix to filter out riders not in MDA
compatible = np.array(maximum_dispatch_radius)

In [10]:
#Compatible becomes an array of 0's and 1's
compatible

array([[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1]])

**Constraints:**



In [11]:
#Ensuring we satisfy MDR
MDR = x @ compatible.T == 1

#Ensuring we constrain x between 1 and 0
rider = x <= 1
driver = x >= 0

#Ensuring our x consists of only one rider
rider_driver = sum(x) == 1

In [12]:
prob = cp.Problem(objective, [MDR,rider,driver,rider_driver])

In [13]:
#Our minimum ETA becomes 0.0542 and we satisfy the given constraints

prob.solve()

0.05424831544228037

In [14]:
#Our x values show that we will be matching rider 1 and driver 8

x.value

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

<h2>Question 2 (25 points) - Batching Dispatch Protocol: Minimize the Aggregated En Route Time Model</h2>
<br>
<font size="+1">
    <ul>
        <li style="color:blue">Recall:</li>
        <br>
        <ul style="color:blue">
            <li>An alternative matching approach is <i>batching</i> where rider requests are collected for a short time window (typically on the order of seconds), at the end of which an optimization problem is solved to pair each request with an open driver.</li>
            <br>
            <li>If there are riders that are not matched in this batch, then they are carried over and used for the optimization problem in the next batching window.</li>
            <br>
            <li>Batching can reduce the rider waiting time relative to the <i>first-dispatch</i> protocol by consolidating requests and making better use of the supply of dispatchable drivers.</li>
            <br>
            <ul>
                <li>Note that the first-dispatch protocol can be viewed as a special case of the batching protocol when each batch consists of exactly one rider request.</li>
                <br>
            </ul>
            <li>Due to its substantial benefits, batching has been implemented widely in major ride-hailing platforms.</li>
            <br>
        </ul>
        <li>Assume you want to implement an improved matching algorithm solution, which is better than the benchmark first-dispatch protocol, that is capable of dispatching multiple drivers, from a collection of available drivers, to a collection of riders.</li>
        <br>
        <ul>
            <li>Note that not every driver will necessarily be dispatched, and not every rider will necessarily be matched with an available driver (it will depend on how the number of riders and number of drivers are related ($\geq, \leq, =$)).</li>
            <br>
        </ul>
        <li>Specifically, you are to implement the so-called <i>batching protocol</i> as an optimization problem given the randomly generated data below.</li>
        <br>
        <li>The data consists of:</li>
        <br>
        <ul>
            <li>A collection of riders that are requesting pick-ups from a collection of drivers.</li>
            <br>
            <li>A data frame consisting of some measure of physical distance from each rider to each driver.</li>
            <br>
            <li>A data frame consisting of the en route time, also known as the expected time of arrival (ETA), which represents the estimated time it will take for a driver to meet a rider once they have been dispatched.</li>
            <br>
        </ul>
        <li><b>Using the given data, formulate an optimization model (decision variables, objective function, constraints) that can be used to implement the <b style="color:blue">batch</b> protocol that seeks to match drivers and riders in a way that <b style="color:blue">minimizes the total en route time (i.e. total ETA).</b></b></li>
        <br>
        <li>That is,</li>
        <br>
        <ul>
            <li>what decisions does the batching protocol have to make?</li>
            <br>
            <li>what is the goal or objective of the batching protocol?</li>
            <br>
            <li>what constraints does the batching protocol have?</li>
            <br>
            <ul>
                <li><i>Hint: it is very possible, depending on your formulation, that you will need slightly different constraints depending on the the relation ($\geq, \leq, =$) between the number of riders and the number of drivers.</i></li>
                <br>
                <li><i>if num_riders $==$ num_drivers:
                    <br>
                    <br>
                    elif num_riders $>$ num_drivers:
                    <br>
                    <br>
                    elif num_riders $<$ num_drivers:
                    <br>
                    <br></i></li>
                <br>
            </ul>
            <li>given the data below, how is the batching protocol suggesting the drivers be dispatched to the riders?</li>
            <br>
        </ul>
        <li style="color:red"><b>You should display your formulation and solutions in an easy to read and interpret format. Your code should be easy to interpret, and if it is not, you should add sufficient commentary through comments, Markdown, or $\LaTeX$.</b></li>
        <br>
    </ul>
</font>

In [15]:
rng = np.random.default_rng(42)

In [16]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances1 = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

MDR = 5 # don't dispatch outside of MDR miles
MDR_1 = (distances1 <= MDR).apply(lambda x: np.where(x, 1, 0))

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta1 = distances1.apply(lambda x: rng.pareto(x)) 

In [17]:
eta1

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,0.063526,0.486780,0.012106,0.027373,0.022479,0.014494,0.118065,6.075330e-01,0.067738,0.944572,0.024951,0.011973,0.013173,0.308573,0.028032,0.169770,0.109341,0.560946
Rider 2,0.248347,0.877538,0.227651,0.015941,0.065569,0.049331,0.019989,3.700551e-02,0.040100,0.200043,0.174144,0.023295,0.310061,0.197105,0.133713,0.032297,0.001956,0.167547
Rider 3,0.001538,0.012287,0.251564,0.037826,0.000859,0.020208,0.218212,4.846250e+06,0.041857,0.121579,0.001143,0.018190,0.098644,0.079491,0.212200,0.530337,0.092002,0.184825
Rider 4,0.067861,0.025888,0.027469,0.067392,0.111707,0.186770,0.672072,2.396151e-02,0.132433,0.015500,0.090009,0.037363,0.114589,0.259361,0.183641,0.045809,0.008773,0.007093
Rider 5,0.004850,0.107339,0.020358,1.008363,0.031519,1.087487,0.004370,3.867961e-02,0.011159,0.071111,1.429397,0.028954,0.045596,0.046097,0.077539,0.039752,0.007066,0.705123
Rider 6,1.001942,0.035147,0.024661,0.009173,0.127556,0.079233,0.060852,5.473993e-02,0.020133,0.054029,0.267101,0.013120,0.277899,0.024043,0.172007,0.033663,0.238994,0.016490
Rider 7,0.003812,0.101547,0.016014,0.065231,0.003376,0.122381,0.051385,5.924328e-03,0.230812,0.049417,0.042624,0.138119,0.293317,0.048901,0.148144,0.000879,0.035532,0.339986
Rider 8,0.091473,26.841782,0.006282,0.198894,0.013723,0.940601,0.340011,1.959335e-01,0.002736,0.273332,0.173345,0.084653,0.153671,0.180374,0.080745,427.269935,0.065623,0.025025
Rider 9,0.043976,0.058538,1.595765,0.000632,0.071684,0.004295,0.030670,1.964556e-02,0.152024,0.019887,0.080725,0.273153,0.160231,0.014484,0.080289,0.043899,0.410271,0.034298
Rider 10,0.682454,0.107257,0.367071,0.019731,0.553485,0.843472,0.027580,8.747272e-03,0.002220,0.053902,0.037098,0.036860,0.038825,0.047305,0.208807,0.149172,0.050306,0.121838


In [18]:
distances1

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


In [19]:
MDR_1

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Rider 2,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Rider 3,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0
Rider 4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
Rider 5,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1
Rider 6,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Rider 7,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0
Rider 8,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
Rider 9,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
Rider 10,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1


**Decision Variables:**

Num_Rider: Keeping dynamic as a part of the length of rows

Num_Drivers: Keeping dynamic as a part of the length of columns

ETA: Minimizing

Matching: Ensuring that we are matching only one rider and driver while minimizing ETA


**Objective Function:**

Minimizing ETA:

f(x) = x1*eta+x2*eta..x18*eta

In [20]:
#Creating a dynamic way to capture changes in drivers and riders
num_drivers = len(eta1.values[0,:])

num_riders = len(eta1.values[:,0])


#Creating decision variables to match dataframe and position based on [row,col]
drivers = cp.Variable(eta1.shape, integer=True)

#Creating objective function as a matrix operation of eta and drivers
total_ETA = sum([eta1.values[i,j]*drivers[i,j] for i in range(eta1.shape[0])
  for j in range(eta1.shape[1])])

#minimizing
objective_function = cp.Minimize(total_ETA)


#creating constraints

cons1 = []
cons2 = []


#scenario 1
if num_riders == num_drivers:
  

  #refers to len of driver, we are working to ensure the sum of each row is equal to one
  #this allows us to match the rider and driver only once
  for i in range(len(eta1.index)):
    cons1.append(sum([drivers[i,j]for j in range(eta1.shape[1])])==1)
  
  
  #refers to len of rider, we are working to ensure the sum of each column is equal to one
  #this allows us to match the rider and driver only once

  for j in range(len(eta1.columns)):
    
    cons2.append(sum([drivers[i,j] for i in range(eta1.shape[0])])==1)


#Scenario 2
elif num_riders > num_drivers:

  #Since number of riders are more than the available drivers we constraint to less than or equal
  #This makes sure that we match only if its optimal!

  for i in range(len(eta1.index)):
    cons1.append(sum([drivers[i,j] for j in range(eta1.shape[1])])<=1)
  
  #Creating columns equal to one to ensure that all drivers are matched with a rider
  #This is a necessary constraint as we know that there is a surplus of riders

  for j in range(len(eta1.columns)):
    cons2.append(sum([drivers[i,j] for i in range(eta1.shape[0])])==1)

  

#Scenario 3
elif num_riders < num_drivers:
  
  #As riders are less than drivers we ensure that each rider is matched
  for i in range(len(eta1.index)):
    cons1.append(sum([drivers[i,j] for j in range(eta1.shape[1])])==1)

  #As drivers are more, we create a less than or equal to allow room for only optimal match
  
  for j in range(len(eta1.columns)):
    cons2.append(sum([drivers[i,j] for i in range(eta1.shape[0])])<=1)

**Constraints:**

In the code above, I am appending each column and row constraint into the lists cons1 and cons2.

Neg and pos : We ensure that all variables are in the range of 0 to 1
Int = True: Satisfies that we are only using integers

In [21]:
neg = [drivers >=0]
pos = [drivers <=1]
constraint = neg+pos+cons1+cons2

**Solution:**

The minimized ETA given the constraints comes out to be 0.1014!

The output array of drivers.value shows the combination of matches among riders and drivers.

In [22]:
prob = cp.Problem(objective_function, constraint)
prob.solve(), drivers.value

(0.10146319689847205,
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,

<h2>Question 3 (25 points) - Batching Dispatch Protocol: Maximize Profit Model with a Maximum Dispatch Radius</h2>
<br>
<font size="+1">
    <ul>
        <li style="color:blue">Recall:</li>
        <br>
        <ul style="color:blue">
            <li>When batch matching is used by ride-hailing platforms, the dispatched driver could miss the opportunity of being matched with a rider that they drive passed while en route to their matched rider.</li>
            <br>
            <li>To mitigate this inefficiency, some ride-hailing platforms proposed a <i>maximum dispatch radius</i> (MDR)  constraint with carefully selected thresholds.</li>
            <br>
            <li>A maximum dispatch radius constraint means that a dispatch is made only if the en route time is below the threshold.</li>
            <br>
            <li>This prevents the driver from being dispatched to pick up very distant riders and experiencing an opportunity cost by not matching with future possible riders that are much closer.</li>
            <br>
        </ul>
        <li>Assume you want to implement a variant on the batching dispatch protocol that minimized total ETA of all matches.</li>
        <br>
        <li><b>This new batching protocol seeks to <b style="color:red">maximize the total profit</b> coming from all matched rider-driver pairs, while ensuring that the matching respects a maximum dispatch radius constraint.</b></li>
        <br>
        <ul>
            <li>Note that not every driver will necessarily be dispatched, and not every rider will necessarily be matched with an available driver (it will depend on how the number of riders and number of drivers are related ($\geq, \leq, =$)).</li>
            <br>
        </ul>
        <li>Specifically, you are to implement a variation on the <i>batching protocol</i> (as described above) as an optimization problem given the randomly generated data below.</li>
        <br>
        <li>The data consists of:</li>
        <br>
        <ul>
            <li>A collection of riders that are requesting pick-ups from a collection of drivers.</li>
            <br>
            <li>A data frame consisting of some measure of physical distance from each rider to each driver.</li>
            <br>
            <li>A maximum dispatch radius (MDR) threshold and a data frame signalling when a rider-driver combination satisfies the MDR constraint.</li>
            <br>
            <li>A data frame consisting of the profits per trip if there is a match between riders and drivers.</li>
            <br>
            <ul>
                <li>Profits can vary based on surge pricing conditions in different locations. That is, surge pricing depends not only on the dynamics of supply and demand, but also on geospatial locations of the rider requests.</li>
                <br>
            </ul>
        </ul>
        <li><b>Using the given data, formulate an optimization model (decision variables, objective function, constraints) that can be used to implement the <b style="color:blue">batch</b> protocol that seeks to match drivers and riders in a way that <b style="color:blue">maximizes the total profits while satisfying the MDR constraints.</b></b></li>
        <br>
        <li>That is,</li>
        <br>
        <ul>
            <li>what decisions does the batching protocol have to make?</li>
            <br>
            <li>what is the goal or objective of the batching protocol?</li>
            <br>
            <li>what constraints does the batching protocol have?</li>
            <br>
            <ul>
                <li><i>Hint: it is very possible, depending on your formulation, that you will need slightly different constraints depending on the the relation ($\geq, \leq, =$) between the number of riders and the number of drivers.</i></li>
                <br>
                <li><i>if num_riders $==$ num_drivers:
                    <br>
                    <br>
                    elif num_riders $>$ num_drivers:
                    <br>
                    <br>
                    elif num_riders $<$ num_drivers:
                    <br>
                    <br></i></li>
                <br>
            </ul>
            <li>given the data below, how is the batching protocol suggesting the drivers be dispatched to the riders?</li>
            <br>
        </ul>
        <li style="color:red"><b>You should display your formulation and solutions in an easy to read and interpret format. Your code should be easy to interpret, and if it is not, you should add sufficient commentary through comments, Markdown, or $\LaTeX$.</b></li>
        <br>
    </ul>
</font>

In [23]:
rng = np.random.default_rng(42)

In [24]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 15 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# need a more realistic way to assign a variance, maybe as a function of distance
profits = distances.apply(lambda x: x + rng.lognormal(mean=1, sigma=3)) 

In [25]:
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


In [26]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1
Rider 2,1,1,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 3,0,0,1,1,0,1,1,1,0,0,0,0,1,0,1,1,0,1
Rider 4,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,0,0,0
Rider 5,0,1,0,1,1,1,0,1,1,0,1,0,1,1,1,0,1,1
Rider 6,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,1,0
Rider 7,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 8,0,1,0,1,0,1,1,1,0,0,1,1,1,0,1,1,0,0
Rider 9,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0
Rider 10,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,1


In [27]:
profits

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,32.262213,15.045836,102.386560,53.660612,659.617014,49.027790,13.326754,9.040791,16.718324,2.488682,277.903786,37.354547,23.076573,10.765658,32.120143,26.864307,23.714480,12.770330
Rider 2,42.420434,2.848371,95.891304,46.349131,662.643925,53.370233,9.799089,13.337420,14.167455,6.258305,256.972496,32.675762,7.141221,20.224299,16.113760,25.051020,21.370928,16.302168
Rider 3,53.386598,25.677189,102.886971,34.507505,660.775926,28.537516,6.021574,2.444497,23.688512,20.519690,274.229813,41.826486,14.101414,17.192115,7.193112,3.506576,20.415064,21.064055
Rider 4,45.375887,24.484224,110.304169,42.465044,657.077275,33.463444,0.948863,15.325151,6.518321,12.830024,278.676943,25.422799,2.083023,8.571396,11.807015,19.928169,17.073940,30.448115
Rider 5,48.348210,13.725864,115.683231,30.866850,640.982423,27.046377,21.695109,16.079946,4.918934,15.605508,257.644214,39.294226,13.718629,11.560516,12.044565,18.979152,11.217353,9.560666
Rider 6,31.958981,30.391188,118.520040,46.848876,648.277159,53.420232,23.386855,23.730335,13.561625,8.741411,255.966580,45.482687,14.007230,6.200780,12.177901,17.447261,5.666159,32.629597
Rider 7,51.174390,23.118147,104.225411,44.676927,657.824000,43.840339,2.557658,14.697851,1.329206,15.393889,262.970688,22.740340,3.436030,17.759216,8.116991,27.824277,17.794809,17.337263
Rider 8,46.146268,2.218374,120.019396,40.326765,663.783117,26.826841,14.624078,16.944839,28.215574,17.726006,267.279533,26.413885,10.281011,15.750051,16.166546,0.719036,25.151733,33.815992
Rider 9,32.626276,18.155342,94.519892,46.024865,648.738074,44.127620,21.834167,25.283054,3.313009,28.054520,259.981271,19.526991,16.979515,11.257548,27.892894,24.318218,9.877142,35.518150
Rider 10,37.146339,16.985972,98.941572,53.938969,645.239295,25.692260,13.077240,31.994896,26.831098,23.032405,279.798626,45.208014,15.899692,9.607751,26.159575,19.920512,11.572707,9.765169


**Decision Variables:**

Number of Riders: Dynamic as of rows

Number of Drivers: Dynamic as of columns

ETA: Time to destination

MDR: Range of acceptable distance

Profits: Combination of distance and time

**Objective Function:**

Maximize total Profit:

f(x)= x1*profit+x2*profit+x3*profit..x18*profit

In [28]:
#Creating dynamic variables for riders and drivers
num_drivers = len(eta1.values[0,:])

num_riders = len(eta1.values[:,0])

#Creating decision variables

drivers = cp.Variable(profits.shape, integer=True)

#Creating the objective function to
#summing the entire matrix and maximizing profits

total_profit = sum([profits.values[i,j]*drivers[i,j] for i in range(profits.shape[0])
  for j in range(profits.shape[1])])

objective_function = cp.Maximize(total_profit)

#Creating lists to append our constraints
cons1 = []


#Scenario 1
if num_riders == num_drivers:


  #refers to len of driver, we are working to ensure the sum of each row is equal to one
  #this allows us to match the rider and driver only once

#Satisfying matching constraint between rider and driver
  for i in range(len(profits.index)):
    cons1.append(sum(drivers[i,j] for j in range(maximum_dispatch_radius.shape[1]))==1)


#Satisfying matching constraint between driver and rider
  for j in range(len(profits.columns)):
    cons1.append(sum(drivers[i,j] for i in range(maximum_dispatch_radius.shape[0]))==1)

#Limiting our total sum of driver and rider matches to the total availability
  cons1.append(sum(maximum_dispatch_radius.values[i,j]*drivers[i,j] for i in range(maximum_dispatch_radius.shape[0]) 
  for j in range(maximum_dispatch_radius.shape[1])) == len(profits.columns))

#Scenario 2
elif num_riders > num_drivers:

  #Since number of riders are more than the available drivers we constraint to less than or equal
  #This makes sure that we match only if its optimal!
  for i in range(len(profits.index)):
    cons1.append(sum(drivers[i,j] for j in range(maximum_dispatch_radius.shape[1]))<=1)

  #Creating columns equal to one to ensure that all drivers are matched with a rider
  #This is a necessary constraint as we know that there is a surplus of riders

  for j in range(len(profits.columns)):
    cons1.append(sum(drivers[i,j] for i in range(maximum_dispatch_radius.shape[0]))==1)
   
#Limiting our total sum of driver and rider matches to the total availability of drivers
#Factoring in that we have more riders then drivers, as I use the length of columns

  cons1.append(sum(maximum_dispatch_radius.values[i,j]*drivers[i,j] for i in range(maximum_dispatch_radius.shape[0]) 
  for j in range(maximum_dispatch_radius.shape[1])) == len(profits.columns))

#Scenario 3
elif num_riders < num_drivers:
  
  #Controlling the rows to equal one creating a match for all riders
  for i in range(len(profits.index)):
    cons1.append(sum(drivers[i,j] for j in range(maximum_dispatch_radius.shape[1]))==1)

  #Controlling the columns to find only optimal matches for driver to rider and therefore 
  #equal or less then one
  for j in range(len(profits.columns)):
    cons1.append(sum(drivers[i,j] for i in range(maximum_dispatch_radius.shape[0]))<=1)

#Limiting our total sum of driver and rider matches to the total availability of riders
#Factoring in that we have more drivers then riders by using length of index

  cons1.append(sum([maximum_dispatch_radius.values[i,j]*drivers[i,j] for i in range(maximum_dispatch_radius.shape[0])
  for j in range(maximum_dispatch_radius.shape[1])]) == len(profits.index))




**Constraints:**

Cons 1: Appending our columns and row constraints based on our driver and rider mix. Creating a independent condition for each loop

Neg + Pos: Ensuring that x is between 0 and 1


In [29]:
neg = [drivers >=0]
pos = [drivers <=1]
constraint = neg+pos+cons1

**Solution:**


The maximum amount of profits we are receiving are 1322.73 given the constraints

Our value matrix shows the driver and rider pairings below

In [30]:
prob = cp.Problem(objective_function, constraint)
prob.solve(), drivers.value

(1322.7378169500578,
 array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 

<h2>Question 4 (25 points) - Optimal Scheduling of Uber Push Notifications</h2>
<br>
<font size="+1">
    <ul>
        <li style="color:red"><b>This problem is not related to optimally matching drivers with riders; it is another problem faced by Uber that is more related to finding an optimal advertising schedule.</b></li>
        <br>
        <ul>
            <li style="color:red">In particular you will be trying to answer the question:</li>
            <br>
            <ul>
                <li style="color:red"><b>What to send when?</b></li>
                <br>
                <li style="color:red">i.e. <b>What push notification to send at what time?</b></li>
                <br>
            </ul>
        </ul>
        <li>The goal of this exercise is for you to read the article <a href="https://www.uber.com/blog/how-uber-optimizes-push-notifications-using-ml/">How Uber Optimizes Push Notifications Using Optimization and Machine Learning</a> and use it as a template to motivate, formulate, and solve your own optimization problem.</li>
        <br>
        <ul>
            <li>It is perfectly okay to try to implement the optimization problem described in the article.</li>
            <br>
        </ul>
        <li><b>The following questions should be answered based off the above article:</b></li>
        <ol>
            <li><b>Describe, in detail, Uber's business motivation for implementing the push notification.</b></li>
            <br>
            <ul>
                <li>This is where you should use your business IQ to motivate the business needs and propose how a prescriptive analytics solution (optimization) can help to address the business needs.</li>
                <br>
                <li>This might require you to do some additional reading of your own, via Google.</li>
                <br>
            </ul>
            <li><b>Clearly identify any points where machine learning would be used in the solution to this push notification scheduling problem. Describe any data that would be required, as well.</b></li>
            <br>
            <li><b>Motivate, formulate, and implement an optimization solution to the push notification scheduling problem using the given simulated data.</b></li>
            <br>
        </ol>
        <li>There is simulated data provided below. The data consists of:</li>
        <br>
        <ul>
            <li>The time periods (time slots) where a push notification can be sent, as well as the number of push notifications that can be sent.</li>
            <br>
            <li>A data frame (scores) consisting of some measure of the value of a <b>(push, time)</b> scheduling pair.</li>
            <br>
        </ul>
        <li style="color:dodgerblue">A few remarks:</li>
        <br>
        <ul style="color:dodgerblue">
            <li>Given a set of candidate push notifications for a user and a set of possible delivery times, the optimization framework should identify the optimal <b>(push, time)</b> pairs.</li>
            <br>
            <li><b>You should ignore all window and expiriation constraints given in the article.</b></li>
            <br>
            <li>Not all notifications need to be scheduled, only the ones that lead to the highest profit.</li>
            <br>
        </ul>
        <li style="color:red"><b>You should display your formulation and solutions in an easy to read and interpret format. Your code should be easy to interpret, and if it is not, you should add sufficient commentary through comments, Markdown, or $\LaTeX$.</b></li>
        <br>
    </ul>
</font>

In [31]:
rng = np.random.default_rng(42)

In [32]:
# Number of times that you can schedule a push notification
time_periods = rng.integers(1,40) # 4

# Number of push notifications you need to schedule
num_notifications = rng.integers(1,40) # 9

# Goal is to schedule NOTIFICATION to TIME PERIOD

# Score that measures the value of a (push, time) pair
# It represents the output of a machine learning model 
# that predicts the probability of a user making an order within 24 hours of receiving push i at time t
scores = pd.DataFrame(rng.uniform(0, 50, (num_notifications, time_periods)),
                         columns=[f'Time Slot {j}' for j in range(1, time_periods+1)],
                         index=[f'Push Notification {i}' for i in range(1,num_notifications+1)])

In [33]:
scores

,Time Slot 1,Time Slot 2,Time Slot 3,Time Slot 4
Push Notification 1,21.943922,42.929896,34.868401,4.708867
Push Notification 2,48.781118,38.056985,39.303215,6.405682
Push Notification 3,22.519297,18.539901,46.338249,32.193256
Push Notification 4,41.138081,22.170710,11.361936,27.729239
Push Notification 5,3.190863,41.381559,31.583220,37.904387
Push Notification 6,17.726298,48.534901,44.656056,38.919175
Push Notification 7,9.731935,23.336050,2.190188,7.714475
Push Notification 8,34.152448,37.238108,48.375487,16.291268
Push Notification 9,18.522985,23.477791,9.473568,6.496075
Push Notification 10,23.785246,11.345467,33.490700,21.857596


# Give your detailed description and motivation of the problem faced by Uber here.

**Describe, in detail, Uber's business motivation for implementing the push notification.**



For customers the value proposition uber eats provides is the wide range of restaurants, pricing and menu information, the estimated time of delivery as well as a wide range of payment options. It creates an easy and highly accessible platform that conveniently delivers food at your request. For restaurants uber eats offers an opportunity to expand their sales growth through delivery, and the app can serve as a marketplace for the restaurant limiting its advertisement through other channeles. Hence, push notifications need to be tailored in a way that helps both restaurants and customers.

Some good features for implementing push notifications can be found in the way that they require little design, it is instant delivery, instant trust through it being sent in-app and one can personalize and tailor the notifications.

Ubers business motivation can be viewed through the lens of demand and revenue. Specifically push notifications can be used for uber eats to deliver information about new restaurants, valuable promotions, new product offerings like alcohol and grocery. Also, the ability to market to the consumer the benefits of being a member.

Sending out push notifications to customers will increase the probability of spending more through the app. Perhaps the customer does not know that he even wants this product, but it is likely a match based on spending patterns. Uber can now  increase customer time in the app and spending, increasing their overall revenue.

Furthermore, by being able to send push notifications about restaurant information, promotions and food, it allows for more restaurants to be seen through the app. For example, a customer is a big fan of mediterranean food, then the customer is shown a wider variety of mediterranean food. More advertisement for the restaurant and more revenue for both parts.



Clearly identify any points where machine learning would be used in the solution to this push notification scheduling problem. Describe any data that would be required, as well.

**Demand:**
Given that we have the availability of customer spending patterns we can create a time series analysis. With that we can predict how a customer will spend at a certain time and therefore choose to send push notifications which will maximize revenue. I assume that we can use this data to generate the push notification score, and with that optimize our decision making.

**Customization:**
With data collected on a customers preference in regards to food, serving size, delivery time we could predict what type of food and restaurant would be the best fit. I see this as a classification problem and I would think we could assemble appropriate features and have the target variables be different types of food/restaurants. After that initial prediction has been made, we could look to historical data on a preference for delivery speed, costs and ingredients. Models to consider are decision trees, logistic regression and knn neighbors.

**Value:**
With the demand and customization element introduced above, the next step would be to use machine learning to predict the value of each push and time pair. I believe this would be on a basis of probability of a user making an order within a given timeframe. As introduced in the article, Uber currently uses and XGBoost model on historical data to predict the conversion probability.

Some of the data we would further need to collect would be time, push and user data.

**Time:** The timing of the push notification, timeframe, expiration date and interaction time. I believe the time element could be found through looking at historical data to understand what time combination is optimal

**Push:** The type of information, class, length, and design that would be optimal for converting the push notification into an interaction. We can gather this based on historical data.

**User data:** A profile of prior purchase history and the main features that we will use in our probability calculation and assigning the value to each push notification.



In [34]:
# Decision Variables

**Decision Variables:**

Send: binary 0/1

Forced decision: Has to pick 0 or 1

Score: Combination of time and ad effectiveness

Time horizon: 4 periods

Amount sent: Fixed to a number in horizon which is 2

**Objective Function:**

Maximize Notification Profit:

F(x) = x1*profit + x2*profit...x31*profit

In [35]:


#Creating variables 
send = cp.Variable(scores.shape, integer=True)


#Creating objective function, which sums matrix and lets me maximize notification profit
notification_profit = sum([scores.values[i,j]*send[i,j] for i in range(scores.shape[0])
  for j in range(scores.shape[1])])


objective_function = cp.Maximize(notification_profit)

**Constraints**:

Pushy: Constraint list that we are appending into

x: Controlling that we are above or equal to 0

y: Controlling that we are equal to or less than 1

Integer = True: Controlling that we are dealing with integers

Row constraint: Appending into list(pushy)

Column constraint: Appending into list(pushy)

In [36]:
# Constraints
pushy = []

x = [send >= 0] #controll that send variable is binary between 0 and 1

y = [send <= 1] #controll that only one notification is sent


#controlling rows: constraining to send at most one of each push notification
for i in range(len(scores.index)):
    pushy.append(sum([send[i,j] for j in range(scores.shape[1])])<= 1)

#controlling columns: #Send at most F(2) pushes in the time horizon
for j in range(len(scores.columns)):
    pushy.append(sum(send[i,j] for i in range(scores.shape[0])) <=2)


In [37]:
constraint = pushy+x+y

In [38]:
# Solution in terms of a scheduling recommendation for the push notifications

**Solution:**

In this solution we are maximizing the profit to be 370.30
as well as satisfying our constraints.

In [39]:
prob = cp.Problem(objective_function, constraint)
prob.solve(), send.value

(370.30952831222953, array([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))